In [1]:
import tweepy
import re
from nltk import *
from nltk.corpus import stopwords

In [2]:
consumer_key = 'BzZbuP9MoZKiJlRGWLt43L529'
consumer_secret = 'UvjqFGMA9RyzIhYLhEqRTTPFWksG6r8eJjVQxbAenbSJNuiie3'
access_token = '1673676428-LHmfijSx0QvqT3vPz7hTWjATZDbCBdR4TkrtqJ9'
access_token_secret = 'UetYT7QN0cPOtmGOpbEilYIESf6haIx47b9BaUl6OKnEP'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [3]:
def get_continuous_chunks(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []
    for i in chunked:
            if type(i) == Tree:
                current_chunk.append(i.label()[:3] + "-")
                current_chunk.append(" ".join([token for token, pos in i.leaves()]))
            elif current_chunk:
                named_entity = " ".join(current_chunk)
                if named_entity not in continuous_chunk:
                    continuous_chunk.append(named_entity)
                    current_chunk = []
            else:
                continue
    return continuous_chunk

In [4]:
def get_relation_tuples(sentences, image):
    relation_tuples = []
    for sentence in sentences:
        rel_tuple_single = []
        rel_tuple_single = get_continuous_chunks(sentence)
        #print(rel_tuple_single)

        if len(rel_tuple_single):
            tokens = word_tokenize(sentence)
            tagged = pos_tag(tokens)

        #extract named entities
        entities = chunk.ne_chunk(tagged)
        #print entities
        for entity in entities:

            if len(entity)>1 and (entity[1] == 'VB' or entity[1] == 'VBD' or entity[1] == 'VBN' or entity[1] == 'VM'):
                if entity[0] not in stopwords.words('english'):
                    rel_tuple_single.append('REL-'+entity[0])
        rel_tuple_single.append('IMG-'+image)

        #print entity
        if len(rel_tuple_single):
            relation_tuples.append(rel_tuple_single)

    with open("input.txt", "a") as myfile:
        myfile.write(str(relation_tuples)+'\n')

In [8]:
pages = ['timesofindia', 'WeirdCrimeFacts', 'CyberCrimeNEWS', 'DNewsCrimeTeam', 'MadisonCrime', 'IowaMurderNews']

for page in pages:
    posts = api.user_timeline(screen_name=page, count = 40)
    for post in posts:
        if 'media' in post.entities:
            text = re.sub(r"http\S+", "", post.text)
            if text:
                try:
                    get_relation_tuples(sent_tokenize(text), post.entities['media'][0]['media_url_https'])
                except:
                    i = 1